## Optimal Power Flow
**Energy Systems Optimization**

by Neha Patankar (last updated: November 16, 2023)

This notebook provides an introduction to the Optimal Power Flow (OPF) problem in electric power systems&mdash;which minimizes the short-run production costs of meeting electricity demand at a number of connected locations from a given set of generators subject to various transmission network flow limit constraints. This will be our first treatment of a *network*, which is critical to all power systems.

We will first introduce a model of transmission flows that assumes we can control the flow along each path, in what is called a "**transport model**." This is a straightforward extension to [Economic Dispatch](04-Economic-Dispatch.ipynb) (ED), where we have multiple supply and demand balance constraints at each location or "node" in the network, and a new set of flow constraints between nodes. This is also similar to other common optimization problems such as fleet routing of shipments.

Full "AC optimal power flow" models are also beyond the scope of this notebook, as the full set of physics associated with the interactions of AC flows introduces non-convexities that make this problem much harder to solve. Due to the non-convex nature of the AC power flow problem, simplified formulations that linearize and approximate these complex constraints are frequently employed in power systems operations, including by electricity system operators, which use a linearized "security-constrained" optimal power flow (which ensures power flows would remain simulatenously feasible across a range of possible contingencies) to clear real-time electricity markets.

We will start off with some simple systems, whose solutions can be worked out manually without resorting to any mathematical optimization model and software. But, eventually we will be solving larger system, thereby emphasizing the importance of such software and mathematical models.

## "Transport" model
We will examine the case where we allow for transmission but ignore the physics of electricity flows, and instead treat it like transporting an ordinary commodity.

$$
\begin{align}
\min \ & \sum_{g \in G} VarCost_g \times GEN_g & \\
\text{s.t.} & \\
 & \sum_{g \in G_i} GEN_g - Demand_i = \sum_{j \in J_i} FLOW_{ij} & \forall \quad i \in \mathcal{N}\\
 & FLOW_{ij} \leq MaxFlow_{ij} & \forall \quad i \in \mathcal{N}, \forall j \in J_i \\
 & FLOW_{ij} = - FLOW_{ji} & \forall \quad i, j \in \mathcal{N} \\
 & GEN_g \leq Pmax_g & \forall \quad g \in G \\
 & GEN_g \geq Pmin_g & \forall \quad g \in G  
\end{align}
$$

We introduce a few new **sets** in the above:
- $\mathcal{N}$, the set of all nodes (or buses) in the network where generation, storage, or demand (load) are located
- $J_i \subset \mathcal{N}$, the subset of nodes that are connected to node $i$
- $G_i \subset G$, the subset of generators located at node $i$
 
The **decision variables** in the above problem are:

- $GEN_{g}$, the generation (in MW) produced by each generator, $g$
- $FLOW_{ij}$, the flow (in MW) along the line from $i$ to $j$

The **parameters** are:

- $Pmin_g$, the minimum operating bounds for the generator (based on engineering or natural resource constraints)
- $Pmax_g$, the maximum operating bounds for the generator (based on engineering or natural resource constraints)
- $Demand_i$, the demand (in MW) at bus $i$
- $MaxFlow_{ij}$, the maximum allowable flow along the line from $i$ to $j$
- $VarCost_g$, the variable cost of generator $g$

Notice how the problem above is equivalent to producing a single type of good at a set of factories and shipping them along capacity-limited corridors (roads, rail lines, etc.) to meet a set of demands in other locations. 

### 1. Load packages

In [1]:
import Pkg; #Pkg.add("VegaLite"); Pkg.add("PrettyTables")
using JuMP, HiGHS
using Plots; plotly();
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
ENV["COLUMNS"]=120; # Set so all columns of DataFrames and Matrices are displayed

### 2. Load and format data

We will load a modified 3-bus case stored in the [MATPOWER case format](https://matpower.org/docs/ref/matpower5.0/caseformat.html). It consists of:

- two generator buses where 1000 MW generators are located, one with variable cost of 50/MWh and another with variable cost of 100/MWh
- one load bus where 600 MW of demand is located
- three lines connecting the buses, each with a maximum flow of 500 MW

The location and numbering of the components:

<img src="img/opf_network.png" style="width: 450px; height: auto" align="left">

In [2]:
datadir = joinpath("OPF_data") 
gen = CSV.read(joinpath(datadir,"gen.csv"), DataFrame);
gencost = CSV.read(joinpath(datadir,"gencost.csv"), DataFrame);
branch = CSV.read(joinpath(datadir,"branch.csv"), DataFrame);
bus = CSV.read(joinpath(datadir,"bus.csv"), DataFrame);

# Rename all columns to lowercase (by convention)
for f in [gen, gencost, branch, bus]
    rename!(f,lowercase.(names(f)))
end

# create generator ids 
gen.id = 1:nrow(gen);
gencost.id = 1:nrow(gencost);

# create line ids 
branch.id = 1:nrow(branch);
# add set of rows for reverse direction with same parameters
branch2 = copy(branch)
branch2.f = branch2.fbus
branch2.fbus = branch.tbus
branch2.tbus = branch2.f
branch2 = branch2[:,names(branch)]
append!(branch,branch2)


# Here are the buses:
bus

Row,bus_i,type,pd,qd
,Int64,Int64,Int64,Float64
1,1,2,0,0.0
2,2,2,0,0.0
3,3,1,600,98.61


Columns pd and qd indicate the [active and reactive power](https://en.wikipedia.org/wiki/AC_power#Active,_reactive,_and_apparent_power) withdrawal at the bus. (We will ignore qd for this notebook, since we are not considering full AC power flows.) We do not need any of other columns for our purposes.

In [3]:
# This is what the generator dataset looks like:
gen

Row,bus,pmax,pmin,id
,Int64,Int64,Int64,Int64
1,1,1000,0,1
2,2,1000,0,2


In [4]:
# and generator cost dataset:
gencost

Row,model,startup,shutdown,n,x1,y1,id
,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2,0,0,2,50,0,1
2,2,0,0,2,100,0,2


In the above, model=2 indicates a polynomial variable cost formulation and the column n=2 indicates that there are two terms. Thus, we have a linear cost (in the x1 column) without any quadratic terms (and a zero constant term):

$$
VarCost_g = x1_g
$$

In [5]:
# Here are the transmission lines:
branch

Row,fbus,tbus,ratea,rateb,ratec,id
,Int64,Int64,Int64,Int64,Int64,Int64
1,1,3,500,500,500,1
2,1,2,500,500,500,2
3,2,3,500,500,500,3
4,3,1,500,500,500,1
5,2,1,500,500,500,2
6,3,2,500,500,500,3


Note that while there are three lines, there are six entries here. There is an entry for each 'direction' of flow the lines can accomodate, hence six entries for three lines. The column `fbus` denotes the ID of the "from bus" or origin bus and the column `tbus` denotes the ID of the "to bus" or destination bus (e.g. `fbus`=1, `tbus`=3 is the flow in the direction from bus 1 to bus 3). 

For this transport model formulation, we are only using transmission line capacity (known as "ratings"), given in `ratea`, `rateb`, and `ratec`. These correspond to different ratings based on how long the line might be overloaded, with `ratec` known as an "emergency rating", which could exceed the long-term rating, `ratea`. We will use `ratea` for this model. The dataset also contains resistance and reactance.

### 3. Create solver function (transport)

In [6]:
#=
Function to solve transport flow problem 
Inputs:
    gen -- dataframe with generator info
    branch -- dataframe with transmission lines info
    gencost -- dataframe with generator info
    bus -- dataframe with bus types and loads
Note: it is always a good idea to include a comment blog describing your
function's inputs clearly!
=#
function transport(gen, branch, gencost, bus)
    Transport = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets
      # Set of all generators
    G = gen.id
      # Set of all nodes
    N = bus.bus_i
      # Note: sets J_i and G_i will be described using dataframe indexing below

    # Decision variables   
    @variables(Transport, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        FLOW[N,N]        # flow
        # Note: flow is not constrained to be positive
        # By convention, positive values will indicate flow from the first to second node
        # in the tuple, and a negative flow will indicate flow from the second to the first
        # This matrix is thus "anti-symmetric", which we will ensure with an appropriate
        # constraint.
    end)
                
    # Objective function: minimize sum of generation variable costs for all generators
    @objective(Transport, Min, 
        sum( gencost[g,:x1] * GEN[g] 
                        for g in G)
    )

    # Supply/demand balance constraints, accounting for power flows in/out of each node
    @constraint(Transport, cBalance[i in N], 
        sum(GEN[g] for g in gen[gen.bus .== i,:id]) 
                - bus[bus.bus_i .== i,:pd][1] ==
        sum(FLOW[i,j] for j in branch[branch.tbus .== i,:fbus]))

    # Max generation constraints
    @constraint(Transport, cMaxGen[g in G],
                    GEN[g] <= gen[g,:pmax])

    # Flow constraints on each branch
    for l in 1:nrow(branch)
        @constraint(Transport, 
            FLOW[branch[l,:fbus][1],branch[l,:tbus][1]] <= 
                        branch[l,:ratea])
    end
    
    # Anti-symmetric flow constraints
    @constraint(Transport, cFlowSymmetric[i in N, j in N],
                    FLOW[i,j] == -FLOW[j,i])

    # Solve statement (! indicates runs in place)
    optimize!(Transport)

    # Dataframe of optimal decision variables
    generation = DataFrame(
        id = gen.id,
        node = gen.bus,
        gen = value.(GEN).data
        )
    
    flows = value.(FLOW).data
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = bus.bus_i,
        value = dual.(cBalance).data)
    
    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        flows,
        prices,
        cost = objective_value(Transport),
        status = termination_status(Transport)
    )
end

transport (generic function with 1 method)

### 4. Solve

In [7]:
solution = transport(gen, branch, gencost, bus)
solution.generation

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 4 cols, 6 nonzeros
1 rows, 3 cols, 2 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-20); columns 0(-11); elements 0(-31) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  3.0000000000e+04
HiGHS run time      :          0.00


Row,id,node,gen
,Int64,Int64,Float64
1,1,1,600.0
2,2,2,0.0


We generate all 600 MW from Gen A at Bus 1.

In [8]:
solution.flows

3×3 Matrix{Float64}:
   -0.0   100.0  500.0
 -100.0    -0.0  100.0
 -500.0  -100.0   -0.0

In turn, the following flows are created: 

- $l_{13}$ = 500 MW
- $l_{12}$ = 100 MW
- $l_{23}$ = 100 MW

Hence, we are able to maximize the capacity of the line from 1 to 3 ($l_{13}$), and then route the remaining power through Bus 2.

### 5. Compare prices

The marginal values of the demand constraints at a given bus represent the change in the objective that results from increasing demand at the bus by one unit. This is the natural definition of a "value" of power at that location, and is the basis for **[locational marginal prices](https://www.iso-ne.com/participate/support/faq/lmp)** (LMPs) found in electricity markets.

In [9]:
solution.prices

Row,node,value
,Int64,Float64
1,1,50.0
2,2,50.0
3,3,50.0


All prices are the same in this case. The interpretation: if we were to add an incremental load at any of the buses, we could meet it from additional production from Gen A which has marginal cost of \$50 / MWh. We are not going to hit any transmission limits.

Note that this system is uncongested, we ignore transmissions losses here, and the lowest cost generator supplies demand at all nodes, leading to equal locational marginal price at all nodes. 